# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-08-24 13:13:34.296292: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-24 13:13:34.544696: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-24 13:13:35.655444: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/Train',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 15000 images belonging to 10 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/Test',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1500 images belonging to 10 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3,activation='relu', input_shape=[128,128,3]))

/home/sayantan/food-classifier-cnn/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1756021490.489105   10974 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4166 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [11]:
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))

In [12]:
cnn.add(tf.keras.layers.Dropout(0.5))

### Step 5 - Output Layer

In [13]:
cnn.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [14]:
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,830,282 (56.57 MB)

 Trainable params: 14,830,282 (56.57 MB)

 Non-trainable params: 0 (0.00 B)

## Part 3 - Training the CNN

### Compiling the CNN

In [15]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [17]:
cnn.fit(x=training_set, validation_data=test_set, epochs=50)

Epoch 1/50


2025-08-24 13:15:40.405158: I external/local_xla/xla/service/service.cc:163] XLA service 0x7504980166b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-24 13:15:40.405184: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2025-08-24 13:15:40.437296: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-24 13:15:40.640869: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91200
2025-08-24 13:15:40.716952: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-08-24 13:15

  2/469 ━━━━━━━━━━━━━━━━━━━━ 25s 54ms/step - accuracy: 0.0859 - loss: 3.9901

I0000 00:00:1756021546.842408   11149 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


155/469 ━━━━━━━━━━━━━━━━━━━━ 28s 89ms/step - accuracy: 0.1127 - loss: 2.5896

...

295/469 ━━━━━━━━━━━━━━━━━━━━ 15s 90ms/step - accuracy: 0.1217 - loss: 2.4655

2025-08-24 13:16:13.573614: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-08-24 13:16:14.527617: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_350', 292 bytes spill stores, 292 bytes spill loads



469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.1320 - loss: 2.4015

/home/sayantan/food-classifier-cnn/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-08-24 13:16:36.998367: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-08-24 13:16:37.715169: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_133', 292 bytes spill stores, 292 bytes spill loads



469/469 ━━━━━━━━━━━━━━━━━━━━ 59s 111ms/step - accuracy: 0.1548 - loss: 2.2814 - val_accuracy: 0.2193 - val_loss: 2.1729
Epoch 2/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 79ms/step - accuracy: 0.2067 - loss: 2.1644 - val_accuracy: 0.2333 - val_loss: 2.0404
Epoch 3/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 79ms/step - accuracy: 0.2659 - loss: 2.0459 - val_accuracy: 0.2900 - val_loss: 1.9453
Epoch 4/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 38s 80ms/step - accuracy: 0.3227 - loss: 1.9361 - val_accuracy: 0.3513 - val_loss: 1.8299
Epoch 5/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 79ms/step - accuracy: 0.3619 - loss: 1.8337 - val_accuracy: 0.3820 - val_loss: 1.7647
Epoch 6/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 79ms/step - accuracy: 0.4016 - loss: 1.7444 - val_accuracy: 0.4220 - val_loss: 1.6651
Epoch 7/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 38s 82ms/step - accuracy: 0.4295 - loss: 1.6701 - val_accuracy: 0.4653 - val_loss: 1.5562
Epoch 8/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 42s 89ms/step - accuracy: 0.4719 - loss: 1.5767 - val_accura

## Part 4 - Saving the Model

In [25]:
cnn.save('model.keras')

## Part 5 - Making a single prediction

In [19]:
import numpy as np
from tensorflow.keras.preprocessing import image

img = image.load_img('dataset/Prediction/Burger-Test.jpeg', target_size=(128,128))
x = image.img_to_array(img) / 255.0
x = np.expand_dims(x, axis=0)
probs = cnn.predict(x)
pred_index = np.argmax(probs[0])
class_indices = training_set.class_indices

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step


In [20]:
print(class_indices)

{'Baked Potato': 0, 'Burger': 1, 'Crispy Chicken': 2, 'Donut': 3, 'Fries': 4, 'Hot Dog': 5, 'Pizza': 6, 'Sandwich': 7, 'Taco': 8, 'Taquito': 9}


In [22]:
if pred_index==0:
    print("Baked Potato")
elif pred_index==1:
    print("Burger")
elif pred_index==2:
    print("Crispy Chicken")
elif pred_index==3:
    print("Donut")
elif pred_index==4:
    print("Fries")
elif pred_index==5:
    print("Hot Dog")
elif pred_index==6:
    print("Pizza")
elif pred_index==7:
    print("Sandwich")
elif pred_index==8:
    print("Taco")
elif pred_index==9:
    print("Taquito")

Burger
